In [ ]:
%%capture
!pip install datasets==1.18.3
# !pip install transformers==4.17.0
!pip install transformers==4.23.1
!pip install jiwer
!pip install huggingsound

Data prep

In [ ]:

speaker = 'M02'

# repo_name = "yip-i/wav2vec2-demo-" + speaker + "-2"
repo_name = "jonatasgrosman/wav2vec2-large-xlsr-53-english"

In [ ]:
import pandas as pd
df = pd.read_csv("output_og.csv")

for index, row in df.iterrows():
    df.at[index, "audio"] = df.at[index, "audio"][1:]
df.to_csv("output.csv")

: 

In [ ]:
data = load_dataset('csv', data_files='output.csv')


: 

In [ ]:
from datasets import load_dataset, load_metric, DatasetDict, Dataset, Audio

data = load_dataset('csv', data_files='output.csv')
data = data.cast_column("audio", Audio(sampling_rate=16_000))
timit = data['train'].filter(lambda x: x == speaker, input_columns=['speaker_id'])


: 

: 

In [ ]:
timit[1]

: 

In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).upper() + " "
    return batch

: 

In [ ]:
timit = timit.map(remove_special_characters)

: 

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
        # print(processor(batch["text"]))
    return batch

: 

In [ ]:
from huggingsound import SpeechRecognitionModel

model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-english")


: 

In [ ]:
audio_paths = [{"path" :"content/downloads/Torgo/M01/Session1/wav_arrayMic/0006.wav",
                "transcription": "Trait"}, 
                {"path" :"content/downloads/Torgo/F01/Session1/wav_arrayMic/0009.wav",
                 "transcription": "GRANDFATHER LIKES TO BE MODERN IN HIS LANGUAGE"}]

transcriptions = model.evaluate(audio_paths)
print(transcriptions)
audio_paths = ["content/downloads/Torgo/M01/Session1/wav_arrayMic/0006.wav", "content/downloads/Torgo/F01/Session1/wav_arrayMic/0009.wav"]
print(model.transcribe(audio_paths))

: 

Create Processor

In [ ]:
# timit = timit.map(prepare_dataset, remove_columns=timit.column_names, num_proc=4)
# timit = timit.filter(lambda x: x < 25 * 16_000, input_columns=["input_length"])


: 

In [ ]:
timit.num_rows

: 

In [ ]:
def prep_asr_testing(torgo_dataset):
  references = []
  print(torgo_dataset.num_rows)
  for i in range(torgo_dataset.num_rows):
    row = {"path": torgo_dataset[i]["audio"]["path"],
           "transcription": torgo_dataset[i]["text"]}
    references.append(row)
  return references


: 

In [ ]:
references = prep_asr_testing(timit)

: 

In [ ]:
references

: 

In [ ]:
evaluation = model.evaluate(references)

: 

: 

Evaluation